# 🎯 Προσωπικός Σύμβουλος Καριέρας με Gemini API

### ✨ Google Colab Notebook με 6 Πράκτορες (Agents):

- 🔍 **Profile Analyzer** – Αναλύει δεξιότητες, ενδιαφέροντα και εμπειρία χρήστη  
- 💼 **Career Suggestion** – Προτείνει κατάλληλες καριέρες βάσει προφίλ  
- 🎓 **Learning Path Generator** – Δημιουργεί προτάσεις εκπαίδευσης με βάση την καριέρα
- 📄 **Resume Advisor** _(με χρήση RAG)_ – Παρέχει feedback σε βιογραφικά
- 🗣️ **Mock Interview** – Προσομοιώνει συνέντευξη και προτείνει πιθανές ερωτήσεις και απαντήσεις  
- 🧭 **Personalized Learning With Gaps** – Δημιουργεί προσωποποιημένο πλάνο εκπαίδευσης και αναφέρει τα κενά που υπάρχουν για την επίτευξη της εκάστοτε καριέρας

### ➕ Επιπλέον Δυνατότητες

- 🔁 **LangGraph DAG Flow** - Οι agents είναι οργανωμένοι σε ροή με εξαρτήσεις μεταξύ τους.

- 💬 **Ιστορικό Συνομιλίας (ChatMemory)** - Διατήρηση μνήμης και ιστορικού για τις συνεδρίες.

- 📥 **Υποστήριξη PDF Βιογραφικού** - Ανάλυση περιεχομένου με `partition_pdf()` για ακριβή εξαγωγή στοιχείων.

- 🧠 **FAISS Embedding Database** - Υλοποίηση Retrieval-Augmented Generation (RAG) για προσωποποιημένες συμβουλές.

- 📊 **Markdown Reporting** - Μορφοποίηση των αποτελεσμάτων σε αναγνώσιμη μορφή με χρήση `markdown.markdown()`.

- 💾 **Προσωποποιημένη Μνήμη (State)** - Διατήρηση learning style, career goals και feedback χρήστη.

- 🌐 **Χρήση Google Gemini API** - Όλοι οι agents αξιοποιούν τη λειτουργία `generate_content()` της πλατφόρμας Gemini.

- 🖥️ **Γραφικό Περιβάλλον (GUI)** - Διαδραστική διεπαφή με χρήση `gradio`.

### Google Drive Conection

In [ ]:
#  # Συνδέει το Google Drive με το περιβάλλον του Google Colab για να αποκτήσουμε πρόσβαση σε αρχεία από τον λογαριασμό μας
#  from google.colab import drive
#  drive.mount('/content/drive')

In [ ]:
#  # Ελέγχω αν έχει πρόσβαση στα αρχεία του Gαoogle Drive
#  !ls /content/drive/MyDrive

### Dependecies Installation

In [ ]:
# !pip install google-generativeai
# !pip install gradio
# !pip install markdown
# !pip install langchain
# !pip install langchain-community
# !pip install langchain-google-genai
# !pip install unstructured
# !pip install pypdf
# !pip install faiss-cpu
# !pip install -q langgraph
# !pip install pdfminer.six
# !pip install "unstructured[all-docs]"

### Imports

In [ ]:
# === Imports ===
import os
import gradio as gr
import markdown
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.tools import tool
from langchain.schema import HumanMessage, AIMessage
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from unstructured.partition.pdf import partition_pdf
from langgraph.graph import StateGraph, END
from typing import TypedDict
import google.generativeai as genai
import json
import re
from google.colab import userdata
from datetime import datetime

PREFS_FILE = "user_personalization.json"
HISTORY_FILE = "/content/drive/MyDrive/career_advisor_files/history.json"

### Api Keys Configuration

In [ ]:
api_key = userdata.get('Gemini_Key')
genai.configure(api_key=api_key)

# Διαθέσιμα Μοντέλα
# print("Διαθέσιμα μοντέλα:")
# for m in genai.list_models():
#     if 'generateContent' in m.supported_generation_methods:
#         print("✅", m.name)

model = genai.GenerativeModel(model_name="models/gemini-1.5-flash-latest")

### Αρχικοποίηση Αρχείου Προτιμήσεων

In [ ]:
# === Αρχικοποίηση Αρχείου Προτιμήσεων ===

# Αρχικοποιεί το αρχείο προτιμήσεων με default τιμές
def init_preferences_file():
    prefs = {
        "learning_style": "visual",
        "career_goals": "Να εργάζομαι εξ αποστάσεως και να έχω ευέλικτο ωράριο."
    }
    with open(PREFS_FILE, "w") as f:
        json.dump(prefs, f, ensure_ascii=False, indent=2)

# Αποθηκεύει τις προτιμήσεις του χρήστη.
def save_user_preferences(learning_style, career_goals):
    with open(PREFS_FILE, "w") as f:
        json.dump({
            "learning_style": learning_style,
            "career_goals": career_goals
        }, f, indent=4, ensure_ascii=False)

# Επιστρέφει τις προτιμήσεις του χρήστη.
def load_personalization_preferences():
    with open(PREFS_FILE, "r") as f:
        prefs = json.load(f)
    return (
        prefs.get("learning_style"),
        prefs.get("career_goals")
    )

# Εκκίνηση του αρχείου αν δεν υπάρχει
if not os.path.exists(PREFS_FILE):
    init_preferences_file()

### Βάση Γνώσης για Βιογραφικά

In [ ]:
# === Resume Knowledge Base ===

# Ανάγνωση περιεχομένου από Google Drive
with open("/content/drive/MyDrive/career_advisor_files/resume_tips.txt", "r", encoding="utf-8") as f:
    resume_tips = f.read()

docs = [Document(page_content=resume_tips)]
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
db = FAISS.from_documents(chunks, embedding)

# === PDF Parser ===
def extract_text_from_pdf(file):
    elements = partition_pdf(file.name)
    return "\n".join([el.text for el in elements if el.text])

### Πράκτορες

##### Ανάλυση Προφίλ

In [ ]:
# === Agent Functions ===

def analyze_profile(text):
    prompt = f"""
      Είσαι ένας πράκτορας που αναλύει προφίλ χρήστη.
      Ανέλυσε το ακόλουθο κείμενο και εντόπισε τις βασικές δεξιότητες, ενδιαφέροντα και γνώσεις.

      {text}

      Επέστρεψε τις δεξιότητες ως λίστα με σύντομη περιγραφή.
      """
    return model.generate_content(prompt).text

#### Πρόταση Καριέρας

In [ ]:
def suggest_careers(skills_summary):
    prompt = f"""
      Με βάση τις παρακάτω δεξιότητες, πρότεινε 2 κατάλληλους επαγγελματικούς ρόλους.
      Για κάθε ρόλο, περιέγραψε:
      - Περιγραφή
      - Απαραίτητες δεξιότητες
      - Γιατί ταιριάζει στον χρήστη

      {skills_summary}
      """
    return model.generate_content(prompt).text

#### Πρόταση Μαθημάτων

In [ ]:
def suggest_learning_path(career_choice):
    prompt = f"""
      Πρότεινε ένα πλάνο εκμάθησης για τον ρόλο: {career_choice}

      Συμπερίλαβε:
      - Τεχνολογίες / έννοιες προς εκμάθηση
      - Online μαθήματα (Coursera, YouTube)
      - Εκτιμώμενος χρόνος ανά θέμα
      """
    return model.generate_content(prompt).text

#### Αξιολόγηση Βιογραφικού

In [ ]:
def suggest_resume_improvements(resume_text):
    relevant_docs = db.similarity_search(resume_text, k=3)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    prompt = f"""
      Αξιολόγησε και πρότεινε βελτιώσεις για το παρακάτω βιογραφικό:

      {resume_text}

      Χρησιμοποιώντας τις παρακάτω οδηγίες:
      {context}

      Περιέγραψε:
      - Τι μπορεί να βελτιωθεί
      - Πώς να βελτιωθεί (μορφή, περιεχόμενο, γλώσσα)
      """
    return model.generate_content(prompt).text

#### Mock Συνέντευξη

In [ ]:
def interview_agent(user_profile, chosen_role, chat_context="", resume_text=None):
    skills_summary = analyze_profile(user_profile + "\n\n" + chat_context)
    rag_context = ""
    if resume_text:
        relevant_docs = db.similarity_search(resume_text, k=3)
        rag_context = "\n\n".join([doc.page_content for doc in relevant_docs])

    prompt = f"""
      Ετοιμάστε mock συνέντευξη με βάση:

      Προφίλ:
      {user_profile}

      Ρόλος:
      {chosen_role}

      Δεξιότητες:
      {skills_summary}

      Βιογραφικό:
      {rag_context}

      Για κάθε ερώτηση, δώσε και μια ενδεικτική καλή απάντηση.

      Παρουσίασε σε μορφή:
      1. Ερώτηση: ...
        Απάντηση: ...
      """

    raw_output = model.generate_content(prompt).text

    # Οργάνωση Αρίθμησης Ερωτήσεων
    def renumber_questions(text):
        pattern = r"(?:^|\n)1\.\s*Ερώτηση:"
        matches = list(re.finditer(pattern, text))
        for i, match in enumerate(matches, 1):
            text = text[:match.start()] + f"\n{i}. Ερώτηση:" + text[match.end()-len("Ερώτηση:"):]
        return text

    return renumber_questions(raw_output)

#### Προτάσεις Μαθημάτων με Personalization

In [ ]:
def personalized_learning_with_gaps(user_skills_text, target_role, personalization_info):
    personalization_info = personalization_info

    prompt_role_desc = f"""
      Δώσε περιγραφή για τον ρόλο: {target_role}
      Περιέγραψε:
      - Τι καθήκοντα έχει
      - Ποιες τεχνολογίες / δεξιότητες απαιτούνται
      """

    role_description = model.generate_content(prompt_role_desc).text

    prompt_gap = f"""
      Σύγκρινε τις παρακάτω δεξιότητες του χρήστη με τις απαιτήσεις του ρόλου:
      --- Δεξιότητες Χρήστη ---
      {user_skills_text}

      --- Περιγραφή Ρόλου ---
      {role_description}

      Βρες:
      - Ποιες δεξιότητες λείπουν;
      - Σε ποιες είναι αρχάριος;
      - Ποια είναι τα σημαντικότερα gaps για να καλύψει;
      Δώσε ως λίστα.
      """

    skill_gaps = model.generate_content(prompt_gap).text

    prompt_learning = f"""
      Ο χρήστης θέλει να γίνει: {target_role}
      Δίνει έμφαση σε:
      - Learning style: {personalization_info['learning_style']}
      - Career goals: {personalization_info['career_goals']}

      Βοήθησέ τον να μάθει ό,τι του λείπει: {skill_gaps}

      Φτιάξε ένα πλάνο εκμάθησης (θεματικό), και δώσε:
      - Τεχνολογίες ή έννοιες προς εκμάθηση
      - Προτάσεις μαθημάτων από Coursera, YouTube
      - Προτεινόμενο χρόνος ανά θέμα
      """

    learning_plan = model.generate_content(prompt_learning).text

    return f"""\
## 🎯 Ρόλος: {target_role}

## 🔎 Ανάλυση Gaps
{skill_gaps}

## 🎓 Προσωποποιημένο Πλάνο Εκμάθησης
{learning_plan}
"""

### LangGraph Agent Flow

In [ ]:
# === LangGraph Agent Flow ===
# Ορίζεται το state του συστήματος, δηλαδή τα δεδομένα που ανταλλάσσονται μεταξύ των πρακτόρων
class AgentState(TypedDict):
    profile: str
    role: str
    resume_text: str
    skills: str
    suggested_roles: str
    learning_plan: str
    resume_feedback: str
    interview_questions: str
    learning_style: str
    career_goals: str

# === Κόμβοι (πράκτορες) ===

def analyze_profile_node(state: AgentState):
    return {"skills": analyze_profile(state["profile"])}

def suggest_roles_node(state: AgentState):
    roles = suggest_careers(state["skills"])
    chosen = state["role"]

    if not chosen:
        match = re.search(r"1\.\s+(.*?)\n", roles)
        if match:
            raw_role = match.group(1).strip()
            chosen = re.sub(r"^[\s:*_]+|[\s:*_]+$", "", raw_role)

    return {
        "suggested_roles": roles,
        "role": chosen or "No role suggested"
    }


def learning_path_node(state: AgentState):
    return {"learning_plan": suggest_learning_path(state["role"])}

def personalized_learning_node(state: AgentState):
    return {
        "learning_plan": personalized_learning_with_gaps(
            user_skills_text=state["skills"],
            target_role=state["role"],
            personalization_info={
                "learning_style": state.get("learning_style", "visual"),
                "career_goals": state.get("career_goals", "")
            }
        )
    }

def resume_feedback_node(state: AgentState):
    if not state["resume_text"]:
        return {"resume_feedback": "(Δεν υποβλήθηκε βιογραφικό)"}
    return {"resume_feedback": suggest_resume_improvements(state["resume_text"])}

def interview_node(state: AgentState):
    return {"interview_questions": interview_agent(state["profile"], state["role"], "", state["resume_text"])}

builder = StateGraph(AgentState)
builder.add_node("AnalyzeProfile", analyze_profile_node)
builder.add_node("SuggestRoles", suggest_roles_node)
builder.add_node("LearningPath", learning_path_node)
builder.set_entry_point("AnalyzeProfile")
builder.add_edge("AnalyzeProfile", "SuggestRoles")
builder.add_edge("SuggestRoles", "LearningPath")
builder.add_edge("LearningPath", END)
default_graph = builder.compile()

builder = StateGraph(AgentState)
builder.add_node("AnalyzeProfile", analyze_profile_node)
builder.add_node("SuggestRoles", suggest_roles_node)
builder.add_node("PersonalizedLearning", personalized_learning_node)
builder.set_entry_point("AnalyzeProfile")
builder.add_edge("AnalyzeProfile", "SuggestRoles")
builder.add_edge("SuggestRoles", "PersonalizedLearning")
builder.add_edge("PersonalizedLearning", END)
personalized_graph = builder.compile()

builder = StateGraph(AgentState)
builder.add_node("AnalyzeProfile", analyze_profile_node)
builder.add_node("SuggestRoles", suggest_roles_node)
builder.add_node("ResumeFeedback", resume_feedback_node)
builder.add_node("Interview", interview_node)
builder.set_entry_point("AnalyzeProfile")
builder.add_edge("AnalyzeProfile", "SuggestRoles")
builder.add_edge("SuggestRoles", "ResumeFeedback")
builder.add_edge("ResumeFeedback", "Interview")
builder.add_edge("Interview", END)
interview_graph = builder.compile()

graph = builder.compile()

### Pipeline

In [ ]:
# === Orchestrator ===
def full_pipeline(
    user_profile,
    chosen_role="",
    resume_file=None,
    chat_history_messages=None,
    mode="default",
    learning_style=None,
    career_goals=None,
    personalized=False
):
    # Χρήσιμο helper για ασφαλή μετατροπή σε string
    def safe_content(content):
        try:
            if hasattr(content, "value"):
                return str(content.value)
            return str(content)
        except Exception as e:
            return f"(Σφάλμα περιεχομένου: {e})"

    # Φορτώνει τις προτιμήσεις του χρήστη
    if learning_style is None or career_goals is None:
        loaded_learning_style, loaded_career_goals = load_personalization_preferences()
        learning_style = learning_style or loaded_learning_style
        career_goals = career_goals or loaded_career_goals

    # Δημιουργία αντικειμένου για αποθήκευση ιστορικού συνομιλίας
    chat_history = ChatMessageHistory()
    memory = ConversationBufferMemory(
      memory_key="chat_history",
      chat_memory=chat_history,
      return_messages=True,
      input_key="input",
      output_key="output"
    )

    # Αν έχει δοθεί προηγούμενο ιστορικό συνομιλίας, το φορτώνει στη μνήμη
    if chat_history_messages:
        for msg in chat_history_messages:
            content = safe_content(msg.get("content", ""))
            if msg['type'] == 'human':
                memory.chat_memory.add_user_message(content)
            elif msg['type'] == 'ai':
                memory.chat_memory.add_ai_message(content)

    # Εξάγει το ιστορικό συνομιλίας σε μορφή κειμένου
    chat_context = memory.load_memory_variables({}).get("history", "")

    # === Επεξεργασία Βιογραφικού ===
    resume_text = ""
    if resume_file:
        try:
            resume_text = extract_text_from_pdf(resume_file)
            user_profile += f"\n\n{resume_text}"
        except Exception as e:
            resume_text = f"(Σφάλμα: {e})"

    # =============================== AGENTS =========================================
    initial_state = {
        "profile": user_profile,
        "resume_text": resume_text,
        "learning_style": learning_style,
        "career_goals": career_goals,
        "role": chosen_role,
        "skills": "",
        "suggested_roles": "",
        "learning_plan": "",
        "resume_feedback": "",
        "interview_questions": ""
    }

    result_markdown = ""
    final_state = {}

    # === PERSONALIZED mode ===
    if mode == "personalized":
        final_state = personalized_graph.invoke(initial_state)
        save_user_preferences(learning_style, career_goals)
        result_markdown = f"""\n{final_state["learning_plan"]}"""

    # === DEFAULT mode ===
    elif mode == "default":
        final_state = default_graph.invoke(initial_state)

        result_markdown = f"""
## 🧠 Ανάλυση Δεξιοτήτων
{final_state['skills']}

## 💼 Προτεινόμενοι Ρόλοι
{final_state['suggested_roles']}

### 🎯 Επιλεγμένος Ρόλος
{final_state['role']}

## 🎓 Προτεινόμενο Πλάνο Εκμάθησης
{final_state['learning_plan']}
"""

    # === INTERVIEW mode ===
    elif mode == "interview":
        final_state = interview_graph.invoke(initial_state)
        parts = []

        if final_state.get("resume_text") and not final_state["resume_text"].startswith("(Σφάλμα"):
            if final_state.get("resume_feedback"):
                parts.append(f"## ✍️ Σχόλια για το Βιογραφικό\n{final_state['resume_feedback']}")

        if final_state.get("interview_questions"):
            parts.append(f"## 🎤 Mock Συνέντευξη\n{final_state['interview_questions']}")

        result_markdown = "\n\n".join(parts)

    else:
        result_markdown = "⚠️ Μη υποστηριζόμενη λειτουργία."

    # ================================================================================

    memory.save_context({"input": user_profile}, {"output": result_markdown})

    session_object = {
        "timestamp": datetime.now().isoformat(timespec="seconds"),
        "mode": mode,
        "role": chosen_role,
        "learning_style": learning_style,
        "career_goals": career_goals,
        "messages": []
    }

    for message in memory.chat_memory.messages:
        if isinstance(message, HumanMessage):
            session_object["messages"].append({"type": "human", "content": safe_content(message.content)})
        elif isinstance(message, AIMessage):
            session_object["messages"].append({"type": "ai", "content": safe_content(message.content)})

    try:
        input_history = chat_history_messages if chat_history_messages else []
        current_messages = session_object["messages"]

        new_messages = []
        for i, msg in enumerate(current_messages):
            if i >= len(input_history) or msg != input_history[i]:
                new_messages.append(msg)

        if new_messages:
            all_sessions = []

            if os.path.exists(HISTORY_FILE):
                with open(HISTORY_FILE, "r", encoding="utf-8") as f:
                    try:
                        all_sessions = json.load(f)
                    except json.JSONDecodeError:
                        all_sessions = []

            changed_session = {
                "timestamp": session_object["timestamp"],
                "mode": mode,
                "role": chosen_role,
                "learning_style": learning_style,
                "career_goals": career_goals,
                "messages": new_messages
            }

            all_sessions.append(changed_session)

            with open(HISTORY_FILE, "w", encoding="utf-8") as f:
                json.dump(all_sessions, f, ensure_ascii=False, indent=2)

    except Exception as e:
        print(f"⚠️ Σφάλμα κατά την αποθήκευση του ιστορικού: {e}")

    return markdown.markdown(result_markdown, extensions=['extra', 'smarty']), session_object["messages"]

### GUI

In [ ]:
# === GUI ===
# Φόρτωση των default τιμών από το JSON αρχείο
default_learning_style, default_career_goals = load_personalization_preferences()

def toggle_personalization_fields(mode):
    show = mode == "personalized"
    if show:
        learning_style, career_goals = load_personalization_preferences()
        return (
            gr.update(visible=True, value=learning_style),
            gr.update(visible=True, value=career_goals)
        )
    return gr.update(visible=show), gr.update(visible=show)

with gr.Blocks(css=".gr-button { width: 100% !important; }") as demo:
    output_html = gr.HTML(label="📊 Αποτελέσματα")

    with gr.Row():
        profile = gr.Textbox(lines=8, label="🔍 Προφίλ / Περιγραφή", interactive=True)

    with gr.Row():
        role = gr.Textbox(lines=1, label="🎯 Επιλεγμένος Ρόλος (προαιρετικά)", interactive=True)

    with gr.Row():
        resume = gr.File(label="📄 Βιογραφικό (PDF)", file_types=[".pdf"])

    with gr.Row():
        mode_dropdown = gr.Dropdown(
            ["default", "interview", "personalized"],
            label="Λειτουργία",
            value="default"
        )

    with gr.Row():
        learning_style_input = gr.Dropdown(
            choices=["visual", "auditory", "hands-on"],
            label="📚 Στυλ Μάθησης",
            value=default_learning_style,
            visible=False
        )

        career_goals_input = gr.Textbox(
            label="🎯 Στόχοι Καριέρας",
            value=default_career_goals,
            visible=False
        )

    mode_dropdown.change(
        toggle_personalization_fields,
        inputs=[mode_dropdown],
        outputs=[learning_style_input, career_goals_input]
    )

    state = gr.State()

    with gr.Row():
        submit_btn = gr.Button("🚀 Submit", scale=1)

    with gr.Row():
        clear_btn = gr.Button("🧹 Clear", scale=1)

    submit_btn.click(
        fn=full_pipeline,
        inputs=[
            profile, role, resume, state,
            mode_dropdown,
            learning_style_input,
            career_goals_input
        ],
        outputs=[output_html, state]
    )

    clear_btn.click(
        fn=lambda: ("", []),
        outputs=[output_html, state]
    )

### Execution

In [ ]:
if __name__ == "__main__":
    demo.launch(share=True, debug=True)